# Availability Groups

***You will need to have followed the steps in the 00-CreateContainers notebook to use this notebook***

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

Setting up availabilty groups is hard

with dbatools it is a little easier :-)

The next block sets the variables for the instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

 Creating connection to the containers


We have a connection to the containers


**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [2]:
Get-DbaDatabase -SqlInstance $SQL2 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

ComputerName : localhost


InstanceName : MSSQLSERVER


SqlInstance  : 31f2db5347e2


Database     : AdventureWorks2017


Status       : Dropped


...

ComputerName : localhost


InstanceName : MSSQLSERVER


SqlInstance  : 31f2db5347e2


Database     : pubs


Status       : Dropped


# Containers

dbatools is not able to create availability groups in containers, so we will create an availability group below using T-SQL. If you are not using containers you can create an availability group with dbatools like this

````
$params = @{
    Primary = $SqlInstances[0]
    PrimarySqlCredential =  $SqlCredential 
    Secondary =   $SqlInstances[1]
    SecondarySqlCredential = $SqlCredential 
    Name = 'TheBeard_AG'
    Database = 'pubs','NorthWind','AdventureWorks2017'
    ClusterType = "None"
    SeedingMode = "Automatic"
    FailoverMode = "Manual"
    Confirm = $false
 }
 
# execute the command
 New-DbaAvailabilityGroup @params
 ````
 
There are many options that you can use here. Read the docs or get our book [beard.media\book]([beard.media\book)

Lets create an availability group to use with this notebook

First we shall create a master key on each instance

In [3]:
New-DbaDbMasterKey -SqlInstance $SQL1 -Credential $SqlCredential -Confirm:$false
New-DbaDbMasterKey -SqlInstance $SQL2 -Credential $SqlCredential -Confirm:$false

ComputerName        : localhost


InstanceName        : MSSQLSERVER


SqlInstance         : b7ee5654561f


Database            : master


CreateDate          : 17/06/2020 19:14:15


...

SqlInstance         : 31f2db5347e2


Database            : master


CreateDate          : 17/06/2020 19:14:15


DateLastModified    : 17/06/2020 19:14:15


IsEncryptedByServer : True


# Certificate

Next we will create a certificate for the endpoints on SQL1, back it up and restore it on SQL2. You can do this for any certificate that you require by the way :-)

In [4]:
New-DbaDbCertificate -SqlInstance $sql1 -Name hadr_cert -Subject hadr_cert 

$EncryptionPassword = ConvertTo-SecureString -AsPlainText "Password4567!!" -force
Backup-DbaDbCertificate -SqlInstance $SQL1 -Suffix $null -Certificate hadr_cert -Path '/var/opt/mssql/backups/' -EncryptionPassword $EncryptionPassword 


ComputerName                 : localhost


InstanceName                 : MSSQLSERVER


SqlInstance                  : b7ee5654561f


Database                     : master


Name                         : hadr_cert


...

InstanceName : MSSQLSERVER


Key          : /var/opt/mssql/backups/\hadr_cert.pvk


Path         : /var/opt/mssql/backups/\hadr_cert.cer


SqlInstance  : b7ee5654561f


Status       : Success


Now we will use `Get-DbaFile` to ge the certificate path and then restore the certificate to SQL2
This will prompt you to ask for input. If you did not wnat this, you can use `-Force` I am also showing verbose output here. All dbatools commands can output verbose information if you use the `-Verbose` switch

In [5]:
$Certificate = (Get-DbaFile -SqlInstance $sql2 -Path '/var/opt/mssql/backups/' -FileType cer).FileName
Restore-DbaDbCertificate -SqlInstance $SQL2 -Path $Certificate -DecryptionPassword $EncryptionPassword  -Verbose


VERBOSE: [20:14:25][Restore-DbaDbCertificate] Processing /var/opt/mssql/backups/\hadr_cert.cer


Confirm


Are you sure you want to perform this action?
Performing the operation "Importing Certificate" on target "hadr_cert on localhost,15593".


[Y] Yes  

[A] Yes to All  

...

Issuer                       : hadr_cert


LastBackupDate               : 01/01/0001 00:00:00


Owner                        : dbo


PrivateKeyEncryptionType     : MasterKey


Serial                       : 56 8c c4 20 69 fe f1 2d


# Endpoints
Now that we have the certificates on each instance, we can create the endpoints using the certificate and start them

In [6]:
New-DbaEndpoint -SqlInstance $sql1 -Name hadr_endpoint -Port 5022 -Certificate hadr_cert 
New-DbaEndpoint -SqlInstance $sql2 -Name hadr_endpoint -Port 5023 -Certificate hadr_cert

Start-DbaEndpoint -SqlInstance $SQL1 -EndPoint hadr_endpoint
Start-DbaEndpoint -SqlInstance $SQL2 -EndPoint hadr_endpoint

ComputerName    : localhost


InstanceName    : MSSQLSERVER


SqlInstance     : b7ee5654561f


ID              : 65536


Name            : hadr_endpoint


...

EndpointType    : DatabaseMirroring


Owner           : sqladmin


IsAdminEndpoint : False


Fqdn            : TCP://Beard-Desktop:5023


IsSystemObject  : False


# Create the Availability Group

we have to use T-SQL to get the right information only because we are using containers

In [7]:

$query = @"
CREATE AVAILABILITY GROUP [TheBeard_AG]
    WITH (CLUSTER_TYPE = NONE)
    FOR REPLICA ON
        N'$($sql1.ComputerNamePhysicalNetBIOS)' WITH (
            ENDPOINT_URL = N'tcp://dbatools_SQL2019_1:5022',
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
                    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    ),
        N'$($sql2.ComputerNamePhysicalNetBIOS)' WITH ( 
		    ENDPOINT_URL = N'tcp://dbatools_SQL2019-1_1:5023', 
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
		    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    );
		
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql1 -Query $query

$query = @"
ALTER AVAILABILITY GROUP [TheBeard_AG] JOIN WITH (CLUSTER_TYPE = NONE);
		 
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql2 -Query $query


we can check that the availability groups on an instance using `Get-DbaAvailabilityGroup`

In [8]:
Get-DbaAvailabilityGroup -SqlInstance $sql1 

ComputerName               : localhost


InstanceName               : MSSQLSERVER


SqlInstance                : b7ee5654561f


LocalReplicaRole           : Primary


AvailabilityGroup          : TheBeard_AG


...

DtcSupportEnabled          : False


AutomatedBackupPreference  : Secondary


AvailabilityReplicas       : {31f2db5347e2, b7ee5654561f}


AvailabilityDatabases      : {}


AvailabilityGroupListeners : {}


Excellent, we have an availablity group - Lets check the replicas

In [9]:
Get-DbaAgReplica -SqlInstance $sql1

ComputerName               : localhost


InstanceName               : MSSQLSERVER


SqlInstance                : b7ee5654561f


AvailabilityGroup          : TheBeard_AG


Name                       : 31f2db5347e2


...

BackupPriority             : 50


EndpointUrl                : tcp://dbatools_SQL2019_1:5022


SessionTimeout             : 10


FailoverMode               : Manual


ReadonlyRoutingList        : {}


We have two replicas - Right now they are not synchronising because there are no databases, You can use this command to see the rol up status for the replicas

Let's check the databases

In [11]:
Get-DbaAgDatabase -SqlInstance $SQL1

There are no databases - Lets add the databases

# Recovery Mode

We need ot set the recovery mode of the databases to full

In [12]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeSystem | Set-DbaDbRecoveryModel -RecoveryModel Full -Confirm:$false

ComputerName   : localhost


InstanceName   : MSSQLSERVER


SqlInstance    : b7ee5654561f


Name           : pubs


Status         : Normal


...

IsAccessible   : True


RecoveryModel  : Full


LastFullBackup : 17/06/2020 18:52:06


LastDiffBackup : 21/12/2019 14:42:00


LastLogBackup  : 01/01/0001 00:00:00


# Backup

Of course, having set the recovery model to full we need to back up the databases otherwise they are in PSUDOSIMPLE mode. You can see this. If we check the recovery model, it will show full

In [13]:
Get-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs

ComputerName   : localhost


InstanceName   : MSSQLSERVER


SqlInstance    : b7ee5654561f


Name           : pubs


Status         : Normal


...

IsAccessible   : True


RecoveryModel  : Full


LastFullBackup : 17/06/2020 18:52:06


LastDiffBackup : 21/12/2019 14:42:00


LastLogBackup  : 01/01/0001 00:00:00


but if we use `Test-DbaDbRecoveryModel` you can see that the *Actual* recovery model is SIMPLE

In [14]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs

ComputerName            : b7ee5654561f


InstanceName            : MSSQLSERVER


SqlInstance             : b7ee5654561f


Database                : pubs


ConfiguredRecoveryModel : FULL


...

InstanceName            : MSSQLSERVER


SqlInstance             : b7ee5654561f


Database                : AdventureWorks2017


ConfiguredRecoveryModel : FULL


ActualRecoveryModel     : SIMPLE


If we backup the databases (like we did in the BackupsandRestores notebook

In [15]:
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Full -IgnoreFileChecks
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Log -IgnoreFileChecks

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 0 %                                                                                   
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 1 %                                                                                   
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 38 %                                                                                  
    [ooooooooooooooooooooooooooooooooo                                                       ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 94 %                                                                                  
    [oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo      ]      
                                                                                                    

                                                                                                    
 Backing up database Northwind to c:\var\opt\mssql\backups\SQL1\Northwind_202006172016.bak          
    Progress: 0 %                                                                                   
    [                                                                                        ]      
                                                                                                    

...

b7ee5654561f Northwind          Full 6.71 MB   Disk       2020-06-17 19:16:01.000 00:00:00 2020-06…


b7ee5654561f pubs               Full 4.52 MB   Disk       2020-06-17 19:16:01.000 00:00:00 2020-06…


b7ee5654561f AdventureWorks2017 Log  84.00 KB  Disk       2020-06-17 19:16:02.000 00:00:00 2020-06…


b7ee5654561f Northwind          Log  80.00 KB  Disk       2020-06-17 19:16:02.000 00:00:00 2020-06…


b7ee5654561f pubs               Log  80.00 KB  Disk       2020-06-17 19:16:02.000 00:00:00 2020-06…


and test our recovery model again

In [16]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs

ComputerName            : b7ee5654561f


InstanceName            : MSSQLSERVER


SqlInstance             : b7ee5654561f


Database                : pubs


ConfiguredRecoveryModel : FULL


...

InstanceName            : MSSQLSERVER


SqlInstance             : b7ee5654561f


Database                : AdventureWorks2017


ConfiguredRecoveryModel : FULL


ActualRecoveryModel     : FULL


All our databases are now in FULL recovery :-)

Let's add them to the Availability Group

In [17]:
Add-DbaAgDatabase -SqlInstance $sql1 -AvailabilityGroup TheBeard_AG -Database AdventureWorks2017, Northwind, pubs -Secondary $sql2 -SeedingMode Automatic

ComputerName         : localhost


InstanceName         : MSSQLSERVER


SqlInstance          : b7ee5654561f


AvailabilityGroup    : TheBeard_AG


Replica              : localhost


...

                                                                                                    
 Trying to add  to localhost,15593                                                                  
    Processing                                                                                      
    [oooooooooooooooooooooooooooooooooooooooooooo                                            ]      
                                                                                                    

                                                                                                    
 Trying to add  to localhost,15593                                                                  
    Processing                                                                                      
    [oooooooooooooooooooooooooooooooooooooooooooooooooooo                                    ]      
                                                                                                    

                                                                                                    
 Trying to add  to localhost,15593                                                                  
    Processing                                                                                      
    [ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo                           ]      
                                                                                                    

                                                                                                    
 Trying to add  to localhost,15593                                                                  
    Processing                                                                                      
    [oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo                  ]      
                                                                                                    

                                                                                                    
 Trying to add  to localhost,15593                                                                  
    Processing                                                                                      
    [ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo         ]      
                                                                                                    

Let's have a look at them


In [18]:
Get-DbaAgDatabase -SqlInstance $SQL1 

ComputerName         : localhost


InstanceName         : MSSQLSERVER


SqlInstance          : b7ee5654561f


AvailabilityGroup    : TheBeard_AG


Replica              : localhost


...

Name                 : pubs


SynchronizationState : Synchronized


IsFailoverReady      : True


IsJoined             : True


IsSuspended          : False


You can look in SSMS if you like :-)

![AG](.\images\ag.png )

## Suspending and resuming data movement

You can use dbatools to suspend and resume data movement for your Availability Group databases. The code below will prompt you to confirm that you wish to suspend them.

In [54]:
Get-DbaAgDatabase -SqlInstance $sql1  | Suspend-DbaAgDbDataMovement


Confirm
Are you sure you want to perform this action?
Performing the operation "Seting availability group  to " on target "".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y



ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True


Confirm
Are you sure you want to perform this action?
Performing the operation "Seting availability group  to " on target "".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y


ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : Northwind
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True


Confirm
Are you sure you want to perform this action?
Performing the operation "Seting availability group  to " on target "".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y


ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : pubs
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True




You can check that they are suspended using `Get-DbaAgDatabase` You will see that the `SynchronizationState` has changed to not synchronising and the `IsSuspended` property is set to true

In [55]:
Get-DbaAgDatabase -SqlInstance $sql1


ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : Northwind
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : pubs
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True




If you want to see if any databases in your Availability Group are not synchronised you can use the code below (if you are looking on an asynchronous replica then you would change Synchronized to Synchronizing.

In [58]:
Get-DbaAgDatabase -SqlInstance $SQL1 | Where SynchronizationState -ne 'Synchronized' | Format-Table


ComputerName InstanceName SqlInstance  AvailabilityGroup Replica   Name               Synchronizati
                                                                                            onState
------------ ------------ -----------  ----------------- -------   ----               -------------
localhost    MSSQLSERVER  da750b72a82d TheBeard_AG       localhost AdventureWorks2017 …ynchronizing
localhost    MSSQLSERVER  da750b72a82d TheBeard_AG       localhost Northwind          …ynchronizing
localhost    MSSQLSERVER  da750b72a82d TheBeard_AG       localhost pubs               …ynchronizing



You can add the capability to resume data movement using dbatools as well with `Resume-DbaAgDbDataMovement` this time we will use `-Confirm:$false` so there are no prompts

In [59]:
Get-DbaAgDatabase -SqlInstance $SQL1| Resume-DbaAgDbDataMovement -Confirm:$false


ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : Northwind
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : pubs
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False




## Removing a database from an Availability Group

You can also remove a database from the availablity group. This code will prompt you to confirm the changing action which is usual dbatools behaviour


In [60]:
Remove-DbaAgDatabase -SqlInstance $SQL1 -AvailabilityGroup TheBeard_AG -Database pubs


Confirm
Are you sure you want to perform this action?
Performing the operation "Removing availability group database [pubs]" on target "localhost,15592".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y



ComputerName      : localhost
InstanceName      : MSSQLSERVER
SqlInstance       : da750b72a82d
AvailabilityGroup : TheBeard_AG
Database          : pubs
Status            : Removed




We can check the user databases on the instances with `Get-DbaDatabase`

In [64]:
Get-DbaDatabase -SqlInstance $SQL1, $SQL2 -ExcludeSystem | Ft


ComputerName InstanceName SqlInstance  Name               Status IsAccessible RecoveryModel LogReus
                                                                                            eWaitSt
                                                                                               atus
------------ ------------ -----------  ----               ------ ------------ ------------- -------
localhost    MSSQLSERVER  da750b72a82d AdventureWorks2017 Normal         True          Full      13
localhost    MSSQLSERVER  da750b72a82d Northwind          Normal         True          Full Nothing
localhost    MSSQLSERVER  da750b72a82d pubs               Normal         True          Full      13



You can see that the pubs database on the secondary is in restoring state.

You can also use `Get-DbaDatabase` to see the Availability Group synchronisation state and the Availability Group that it is joined to by getting some of the properties that are not displayed by default

In [67]:
Get-DbaDatabase -SqlInstance $SQL1, $SQL2 -ExcludeSystem | Select SqlInstance, Name, AvailabilityDatabaseSynchronizationState, AvailabilityGroupName 


SqlInstance  Name               AvailabilityDatabaseSynchronizationState AvailabilityGroupName
-----------  ----               ---------------------------------------- ---------------------
da750b72a82d AdventureWorks2017                             Synchronized TheBeard_AG
da750b72a82d Northwind                                      Synchronized TheBeard_AG
da750b72a82d pubs                                                        
c134b2316f63 AdventureWorks2017                            Synchronizing TheBeard_AG
c134b2316f63 Northwind                                     Synchronizing TheBeard_AG
c134b2316f63 pubs                                                        



You can see that the pubs database is not joined to the Availability Group

# Failover

You can use dbatools to failover the Availability Group. Lets take a look at the Availability Group

In [68]:
Get-DbaAvailabilityGroup -SqlInstance $SQL1


ComputerName               : localhost
InstanceName               : MSSQLSERVER
SqlInstance                : da750b72a82d
LocalReplicaRole           : Primary
AvailabilityGroup          : TheBeard_AG
PrimaryReplica             : da750b72a82d
ClusterType                : None
DtcSupportEnabled          : False
AutomatedBackupPreference  : Secondary
AvailabilityReplicas       : {c134b2316f63, da750b72a82d}
AvailabilityDatabases      : {AdventureWorks2017, Northwind}
AvailabilityGroupListeners : {}




The primary replica for the Availability Group is SQL1. Lets fail it over to SQL2

In [70]:
 Invoke-DbaAgFailover -SqlInstance $SQL2 -AvailabilityGroup TheBeard_AG


Confirm
Are you sure you want to perform this action?
Performing the operation "Gracefully failing over TheBeard_AG" on target "localhost,15593".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y


It has prompted us for confirmation and then told us that the cluster type that we have used for the containers does not support this type of failover so we need to use the force                     parameter

In [71]:
 Invoke-DbaAgFailover -SqlInstance $SQL2 -AvailabilityGroup TheBeard_AG -Force


ComputerName               : localhost
InstanceName               : MSSQLSERVER
SqlInstance                : c134b2316f63
LocalReplicaRole           : Primary
AvailabilityGroup          : TheBeard_AG
PrimaryReplica             : c134b2316f63
ClusterType                : None
DtcSupportEnabled          : False
AutomatedBackupPreference  : Secondary
AvailabilityReplicas       : {c134b2316f63, da750b72a82d}
AvailabilityDatabases      : {AdventureWorks2017, Northwind}
AvailabilityGroupListeners : {}




When we check the Availability Group on SQL1 again

In [72]:
Get-DbaAvailabilityGroup -SqlInstance $SQL1


ComputerName               : localhost
InstanceName               : MSSQLSERVER
SqlInstance                : da750b72a82d
LocalReplicaRole           : Primary
AvailabilityGroup          : TheBeard_AG
PrimaryReplica             : da750b72a82d
ClusterType                : None
DtcSupportEnabled          : False
AutomatedBackupPreference  : Secondary
AvailabilityReplicas       : {c134b2316f63, da750b72a82d}
AvailabilityDatabases      : {AdventureWorks2017, Northwind}
AvailabilityGroupListeners : {}




You can see by comparing the output that the primary replica has swapped to the other container. I know it is a little hard to see with container names

In [73]:
Get-DbaDatabase -SqlInstance $SqlInstances -SqlCredential $SqlCredential -ExcludeSystem | Select SqlInstance, Name, AvailabilityDatabaseSynchronizationState, AvailabilityGroupName 


SqlInstance  Name               AvailabilityDatabaseSynchronizationState AvailabilityGroupName
-----------  ----               ---------------------------------------- ---------------------
da750b72a82d AdventureWorks2017                         NotSynchronizing TheBeard_AG
da750b72a82d Northwind                                  NotSynchronizing TheBeard_AG
da750b72a82d pubs                                                        
c134b2316f63 AdventureWorks2017                             Synchronized TheBeard_AG
c134b2316f63 Northwind                                      Synchronized TheBeard_AG
c134b2316f63 pubs                                                        



In [74]:
Get-DbaAgDatabase -SqlInstance $sql2  | Suspend-DbaAgDbDataMovement


Confirm
Are you sure you want to perform this action?
Performing the operation "Seting availability group  to " on target "".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  a



ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : c134b2316f63
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : c134b2316f63
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : Northwind
SynchronizationState : NotSynchronizing
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : True




In [75]:
Get-DbaAgDatabase -SqlInstance $sql1  | Resume-DbaAgDbDataMovement


Confirm
Are you sure you want to perform this action?
Performing the operation "Seting availability group  to " on target "".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  a



ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : NotSynchronizing
IsFailoverReady      : False
IsJoined             : True
IsSuspended          : False

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : da750b72a82d
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : Northwind
SynchronizationState : NotSynchronizing
IsFailoverReady      : False
IsJoined             : True
IsSuspended          : False




In [77]:
Get-DbaDatabase -SqlInstance $SqlInstances -SqlCredential $SqlCredential -ExcludeSystem | Select SqlInstance, Name, AvailabilityDatabaseSynchronizationState, AvailabilityGroupName 


SqlInstance  Name               AvailabilityDatabaseSynchronizationState AvailabilityGroupName
-----------  ----               ---------------------------------------- ---------------------
da750b72a82d AdventureWorks2017                         NotSynchronizing TheBeard_AG
da750b72a82d Northwind                                  NotSynchronizing TheBeard_AG
da750b72a82d pubs                                                        
c134b2316f63 AdventureWorks2017                         NotSynchronizing TheBeard_AG
c134b2316f63 Northwind                                  NotSynchronizing TheBeard_AG
c134b2316f63 pubs                                                        



# Clean Up

The 99-CleanUpContainers notebook will remove the containers, files and directory - it will leave the image so you do not have to download it again!